In [4]:
import glob

files = glob.glob('GermInnC/**/*.txt')

In [10]:
files[:100]

['GermInnC/1901-1950/NEWS_P6_OOD_1915_Innsbrucker Nachrichten_RAW.txt',
 'GermInnC/1901-1950/NEWS_P6_OMD_1928_Arbeiterstimme_RAW.txt',
 'GermInnC/1901-1950/HUMA_P6_WMD_1920_Evangelischer Sozialismus_RAW .txt',
 'GermInnC/1901-1950/DRAM_P6_WMD_1915_Heimkehr_RAW.txt',
 'GermInnC/1901-1950/NEWS_P6_OOD_1914_Wiener Zeitung_RAW.txt',
 'GermInnC/1901-1950/SCIE_P6_OMD_1901_Dampf_RAW.txt',
 'GermInnC/1901-1950/LEGA_P6_OMD_1922_ThÅringerKirchenblatt_RAW.txt',
 'GermInnC/1901-1950/HUMA_P6_WMD_1920_Werden und Leben der Staaten_RAW.txt',
 'GermInnC/1901-1950/NEWS_P6_WOD_1936_Liechtensteiner Vaterland_RAW.txt',
 'GermInnC/1901-1950/HUMA_P6_WMD_1901_Geschichte des Hexenwahns_1_RAW.txt',
 'GermInnC/1901-1950/LEGA_P6_OOD_1920_GesetzBauernkammern_RAW.txt',
 'GermInnC/1901-1950/NEWS_P6_OMD_1945_Leipziger Neueste Nachrichten_RAW.txt',
 'GermInnC/1901-1950/SERM_P6_WMD_1917_Den scheidenden Glocken_RAW.txt',
 'GermInnC/1901-1950/SCIE_P6_WMD_1907_Einfluss ErnÑhrung_1RAW.txt',
 'GermInnC/1901-1950/NEWS_P6_WOD_

In [28]:
import os
import pandas as pd

data = []
error = 0
for file in files:
    genre, period, region, year, title = os.path.basename(file).split('_')[:5]
    try: 
        with open(file, 'r', encoding='UTF-8') as f:
            text = f.read()
    except Exception as e:
        error += 1
        print(e)
        continue

    entry = {
        'text': text,
        'title':title,
        'genre': genre,
        'period': period,
        'region': region,
        'year': year
    }
    data.append(entry)


'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
'utf-8' codec can't decode byte 0xfc in position 97: invalid start byte
'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
'utf-8' codec can't decode byte 0xfc in position 23: invalid start byte
'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
'utf-8' codec can't decode byte 0xfc in position 12: invalid start byte
'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
'ut

In [29]:
error

368

In [14]:
from pprint import pprint
pprint(parts[:100])

[['NEWS', 'P6', 'OOD', '1915', 'Innsbrucker Nachrichten', 'RAW.txt'],
 ['NEWS', 'P6', 'OMD', '1928', 'Arbeiterstimme', 'RAW.txt'],
 ['HUMA', 'P6', 'WMD', '1920', 'Evangelischer Sozialismus', 'RAW .txt'],
 ['DRAM', 'P6', 'WMD', '1915', 'Heimkehr', 'RAW.txt'],
 ['NEWS', 'P6', 'OOD', '1914', 'Wiener Zeitung', 'RAW.txt'],
 ['SCIE', 'P6', 'OMD', '1901', 'Dampf', 'RAW.txt'],
 ['LEGA', 'P6', 'OMD', '1922', 'ThÅringerKirchenblatt', 'RAW.txt'],
 ['HUMA', 'P6', 'WMD', '1920', 'Werden und Leben der Staaten', 'RAW.txt'],
 ['NEWS', 'P6', 'WOD', '1936', 'Liechtensteiner Vaterland', 'RAW.txt'],
 ['HUMA', 'P6', 'WMD', '1901', 'Geschichte des Hexenwahns', '1', 'RAW.txt'],
 ['LEGA', 'P6', 'OOD', '1920', 'GesetzBauernkammern', 'RAW.txt'],
 ['NEWS', 'P6', 'OMD', '1945', 'Leipziger Neueste Nachrichten', 'RAW.txt'],
 ['SERM', 'P6', 'WMD', '1917', 'Den scheidenden Glocken', 'RAW.txt'],
 ['SCIE', 'P6', 'WMD', '1907', 'Einfluss ErnÑhrung', '1RAW.txt'],
 ['NEWS', 'P6', 'WOD', '1913', 'Staufener Wochenblatt', 'R

In [ ]:
for 